## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
#%matplotlib notebook

from brainpipe.system import study
from brainpipe.feature import TF
from brainpipe.visual import *
from brainpipe.statistics import *

In [2]:
sf = 512
n_perm = 1000
f = (1,120,4,2)  # Frequency vector: (from, to, width, step) 1 to 120 Hz 4 2 #0.1, 13, 0.5, 0.1
baseline = [0, 700] # Time in seconds (rest 700ms au milieu des 1s extraite)
width, step = 50, 10 #Time vector

### Compute TF on unbalanced data for all selected electrodes
#### No baseline correction - over 5.5s after odor is sent

In [4]:
st = study('Olfacto_bad_good_poor_partial_detailed')
path_data = path.join (st.path, 'database/Encoding_EpiPerf_4500_expi_noart/')
path2save = path.join (st.path, 'feature/1_TF_Encoding_EpiPerf_4500_expi_noart/')
conds = ['poor', 'partial','detailed']
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ'] #LEFC','CHAF','VACJ','SEMC','FERJ','MICP','PIRJ

for su in subjects:
    data0 = np.load(path.join(path_data, su+'_odor_poor_bipo_sel.npz'))['x']
    data1 = np.load(path.join (path_data, su+'_odor_partial_bipo_sel.npz'))['x']
    names = np.load(path.join(path_data, su+'_odor_partial_bipo_sel.npz'))['label']
    channels = np.load(path.join(path_data, su+'_odor_partial_bipo_sel.npz'))['channel']
    
    filename2 = path.join (path_data, su+'_odor_detailed_bipo_sel.npz')
    if path.isfile(filename2) == False:
        print (su, 'poor shape: ', data0.shape, 'partial shape: ', data1.shape)
        # ========================= COMPUTE TF FOR 1 ELEC =============================================
        for elec in range(data0.shape[0]):
            poor, partial = data0[elec,:,:], data1[elec,:,:]
            channel, label, sf = channels[elec], names[elec], 512
            print (su, channel, label, poor.shape, partial.shape, 'nb points', poor.shape[0])

            # time = all data points in the original data (window to plot is selected afterwards)
            time = 1000 * np.arange(poor.shape[0]) / sf #to set the 0 in your TF
            print ('Time points: ', len(time),min(time), max(time))
            tfObj = TF(sf, poor.shape[0], f=f, time=time, width=width, step=step,baseline=baseline, norm=3)
            xtf_poor, _ = tfObj.get((poor), statmeth=None, n_perm=n_perm, maxstat=False, tail=2, metric='m_center')
            xtf_partial, _ = tfObj.get((partial), statmeth=None, n_perm=n_perm, maxstat=False, tail=2, metric='m_center')
            print ('TF size :', xtf_poor.shape, xtf_partial.shape)
            xtf_poor, xtf_partial = 100*np.swapaxes(xtf_poor,0,1), 100*np.swapaxes(xtf_partial,0,1)

            # plot both TF in the same figure
            xtf_all = np.concatenate((xtf_poor, xtf_partial), axis=0)
            print ('TF size :', xtf_poor.shape,xtf_partial.shape)
            fig = plt.figure(figsize=(10, 4))
            title = su+' Elec('+str(elec)+') '+channel+' '+label+' Epi Perf'
            fig, _ = tfObj.plot2D(fig, xtf_all, cmap='viridis', xvec=tfObj.xvec,
                         yvec=tfObj.yvec, xlabel='Time (ms)', vmin=-100, vmax=100, ycb=-20,
                         ylabel='Frequency (hz)', figtitle=title, title=['Poor EM', 'Partial EM'],
                         cblabel='Power modulations (%)', pltype='imshow', resample=(0.1, 0.1),
                        sharex=False, sharey=False, subdim=(1,2), subspace={'top':0.8})
            #Save all your plots
            fname = (path2save+su+'_TF_elec_('+str(elec)+')_'+channel+'_'+label+'_Poor-Partial.png')
            print (fname)
            fig.savefig(fname, dpi=300, bbox_inches='tight')
            plt.close()
        
    if path.isfile(filename2) == True:
        data2 = np.load(filename2)['x']
        print (su, 'poor shape: ', data0.shape, 'partial shape: ', data1.shape, 'detailed shape', data2.shape)
        # ========================= COMPUTE TF FOR 1 ELEC =============================================
        for elec in range(data0.shape[0]):
            poor, partial, detailed = data0[elec,:,:], data1[elec,:,:], data2[elec,:,:]
            channel, label, sf = channels[elec], names[elec], 512
            print (su, channel, label, poor.shape, partial.shape, detailed.shape, 'nb points', poor.shape[0])

            # time = all data points in the original data (window to plot is selected afterwards)
            time = 1000 * np.arange(poor.shape[0]) / sf #to set the 0 in your TF
            print ('Time points: ', len(time),min(time), max(time))
            tfObj = TF(sf, poor.shape[0], f=f, time=time, width=width, step=step,baseline=baseline, norm=3)
            xtf_poor, _ = tfObj.get((poor), statmeth=None, n_perm=n_perm, maxstat=False, tail=2, metric='m_center')
            xtf_partial, _ = tfObj.get((partial), statmeth=None, n_perm=n_perm, maxstat=False, tail=2, metric='m_center')
            xtf_detailed, _ = tfObj.get((detailed), statmeth=None, n_perm=n_perm, maxstat=False, tail=2, metric='m_center')
            print ('TF size :', xtf_poor.shape, xtf_partial.shape,xtf_detailed.shape)
            xtf_poor, xtf_partial, xtf_detailed = 100*np.swapaxes(xtf_poor,0,1), 100*np.swapaxes(xtf_partial,0,1), 100*np.swapaxes(xtf_detailed,0,1)

            # plot both TF in the same figure
            xtf_all = np.concatenate((xtf_poor, xtf_partial, xtf_detailed), axis=0)
            print ('TF size :', xtf_poor.shape,xtf_partial.shape, xtf_detailed.shape, xtf_all.shape)
            fig = plt.figure(elec, figsize=(15, 5))
            title = su+' Elec('+str(elec)+') '+channel+' '+label+' Epi Perf'
            fig, _ = tfObj.plot2D(fig, xtf_all, cmap='viridis', xvec=tfObj.xvec,
                         yvec=tfObj.yvec, xlabel='Time (ms)', vmin=-100, vmax=100, ycb=-20,
                         ylabel='Frequency (hz)', figtitle=title, title=['Poor EM', 'Partial EM','Detailed EM'],
                         cblabel='Power modulations (%)', pltype='imshow', resample=(0.1, 0.1),
                        sharex=False, sharey=False, subdim=(1,3), subspace={'top':0.8})
            #Save all your plots
            fname = (path2save+su+'_TF_elec_('+str(elec)+')_'+channel+'_'+label+'_Poor-Partial-Detailed.png')
            print (fname)
            fig.savefig(fname, dpi=300, bbox_inches='tight')
            plt.close()
        del poor, partial, detailed, sf, channel



-> Olfacto_bad_good_poor_partial_detailed loaded
LEFC poor shape:  (148, 2304, 23) partial shape:  (148, 2304, 20) detailed shape (148, 2304, 14)
LEFC a2-a1 Amg-pPirT (2304, 23) (2304, 20) (2304, 14) nb points 2304
Time points:  2304 -3005.859375 1492.1875
TF size : (58, 1, 226) (58, 1, 226) (58, 1, 226)
TF size : (1, 58, 226) (1, 58, 226) (1, 58, 226) (3, 58, 226)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto_bad_good_poor_partial_detailed/feature/1_TF_Encoding_EpiPerf_4500_expi_noart/LEFC_TF_elec_(0)_a2-a1_Amg-pPirT_Poor-Partial-Detailed.png
LEFC a6-a5 Ins-Ppo (2304, 23) (2304, 20) (2304, 14) nb points 2304
Time points:  2304 -3005.859375 1492.1875
TF size : (58, 1, 226) (58, 1, 226) (58, 1, 226)
TF size : (1, 58, 226) (1, 58, 226) (1, 58, 226) (3, 58, 226)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto_bad_good_poor_partial_detailed/feature/1_TF_Encoding_EpiPerf_4500_expi_noart/LEFC_TF_elec_(1)_a6-a5_Ins-Ppo_Poor-Partial-Detailed.png
LEFC a7-a6 Ins-Ppo (2304, 23) (2

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f4f461a8488>
Traceback (most recent call last):
  File "/home/karim/anaconda3/lib/python3.5/weakref.py", line 109, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt


TF size : (58, 1, 226) (58, 1, 226) (58, 1, 226)
TF size : (1, 58, 226) (1, 58, 226) (1, 58, 226) (3, 58, 226)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto_bad_good_poor_partial_detailed/feature/1_TF_Encoding_EpiPerf_4500_expi_noart/LEFC_TF_elec_(2)_a7-a6_Ins-Ppo_Poor-Partial-Detailed.png
LEFC a8-a7 aSTG-aMTG&Ins-Ppo (2304, 23) (2304, 20) (2304, 14) nb points 2304
Time points:  2304 -3005.859375 1492.1875
TF size : (58, 1, 226) (58, 1, 226) (58, 1, 226)
TF size : (1, 58, 226) (1, 58, 226) (1, 58, 226) (3, 58, 226)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto_bad_good_poor_partial_detailed/feature/1_TF_Encoding_EpiPerf_4500_expi_noart/LEFC_TF_elec_(3)_a8-a7_aSTG-aMTG&Ins-Ppo_Poor-Partial-Detailed.png
LEFC a9-a8 aSTG-aMTG (2304, 23) (2304, 20) (2304, 14) nb points 2304
Time points:  2304 -3005.859375 1492.1875
TF size : (58, 1, 226) (58, 1, 226) (58, 1, 226)
TF size : (1, 58, 226) (1, 58, 226) (1, 58, 226) (3, 58, 226)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Ol

Process ForkPoolWorker-2175:
Process ForkPoolWorker-2176:
Process ForkPoolWorker-2173:
Traceback (most recent call last):
Process ForkPoolWorker-2171:
Process ForkPoolWorker-2172:
  File "/home/karim/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
Process ForkPoolWorker-2174:
Process ForkPoolWorker-2170:
Traceback (most recent call last):
  File "/home/karim/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/karim/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/karim/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/karim/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootst

KeyboardInterrupt: 